#### 인입선 데이터 처리
* 여기서는 인입지점의 좌표정보는 처리하지 않음

In [23]:
import pandas as pd
from freeman.aiddd.data_manager import read_data, write_data, common_columns

In [24]:
dtype = {'cons_id': str}
df_cons = read_data(
    '1e-merge-cons-pole-line-path', process_seq='4th', dtype=dtype,
)
df_sl = read_data('SL_DATA', data_type='provide', process_seq='3rd')

df_cons.shape, df_sl.shape

/home/freeman/projects/ai-projects/utils/freeman/aiddd/data_manager.py:31: DtypeWarning: Columns (135) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(full_path, **kwargs)


((11383, 139), (22632, 57))

#### 전처리

In [25]:
# df_cons[?????_comp_id]에 NaN값이 존재(읽어오면서 생긴듯)
df_cons.fillna('', inplace=True)

In [26]:
training_columns = [
    '공사번호', '전산화번호', '인입전선종류코드', 
    '고객공급선규격코드', '지지물간거리', '조수'
]
rename_columns = {
    '공사번호': common_columns['공사번호'],
    '전산화번호': common_columns['전산화번호'],
    '인입전선종류코드': common_columns['인입전선종류코드'],
    '고객공급선규격코드': common_columns['고객공급선규격코드'],
    '지지물간거리': common_columns['지지물간거리'],
    '조수': common_columns['조수'],
}
print(rename_columns)

df_training = df_sl[training_columns].copy()
df_training.rename(columns=rename_columns, inplace=True)
df_training.shape

{'공사번호': 'cons_id', '전산화번호': 'comp_id', '인입전선종류코드': 'sl_type_cd', '고객공급선규격코드': 'sl_spec_cd', '지지물간거리': 'span', '조수': 'supervisor'}


(22632, 6)

In [27]:
df_training = df_training[df_training.cons_id.isin(df_cons.cons_id)]
df_training.shape

(11892, 6)

In [28]:
# 결측치 확인
df_training.isna().sum()

cons_id       0
comp_id       0
sl_type_cd    0
sl_spec_cd    0
span          0
supervisor    0
dtype: int64

In [29]:
df_training.cons_id.value_counts()

cons_id
474220224975    8
477220213640    7
70HC20213607    6
70HC20225986    6
474220223607    6
               ..
477220233923    1
477220233924    1
477220233925    1
477220233928    1
474620234659    1
Name: count, Length: 11209, dtype: int64

In [30]:
[col for col in df_cons.columns if '_cnts' in col]

['line_cnts', 'pole_cnts', 'sl_cnts', 'real_pole_cnts', 'support_pole_cnts']

#### 전처리 sl 데이터 저장

In [31]:
write_data('1f-preprocessed-sl', df_training, process_seq='4th')

In [32]:
df_training.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11892 entries, 17 to 22631
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   cons_id     11892 non-null  object 
 1   comp_id     11892 non-null  object 
 2   sl_type_cd  11892 non-null  object 
 3   sl_spec_cd  11892 non-null  float64
 4   span        11892 non-null  int64  
 5   supervisor  11892 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 650.3+ KB


In [33]:
df_training = pd.get_dummies(
    df_training,
    columns=['sl_type_cd', 'sl_spec_cd'],
    prefix=['sl_type', 'sl_spec']
)

In [34]:
df_training.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11892 entries, 17 to 22631
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   cons_id        11892 non-null  object
 1   comp_id        11892 non-null  object
 2   span           11892 non-null  int64 
 3   supervisor     11892 non-null  int64 
 4   sl_type_C1     11892 non-null  bool  
 5   sl_type_C2     11892 non-null  bool  
 6   sl_type_C4     11892 non-null  bool  
 7   sl_type_D2     11892 non-null  bool  
 8   sl_type_D3     11892 non-null  bool  
 9   sl_type_D4     11892 non-null  bool  
 10  sl_type_OW     11892 non-null  bool  
 11  sl_type_V2     11892 non-null  bool  
 12  sl_spec_1.6    11892 non-null  bool  
 13  sl_spec_2.6    11892 non-null  bool  
 14  sl_spec_3.2    11892 non-null  bool  
 15  sl_spec_5.5    11892 non-null  bool  
 16  sl_spec_10.0   11892 non-null  bool  
 17  sl_spec_16.0   11892 non-null  bool  
 18  sl_spec_22.0   11892 non-null 

In [35]:
# bool형을 int로 변환
df_training = df_training.applymap(
    lambda x: int(x) if isinstance(x, bool) else x
)

In [36]:
df_training[df_training.cons_id=='474220224975']
# 당연하지만 인입선의 전원측전산화번호가 2개인 경우도 있음

,cons_id,comp_id,span,supervisor,sl_type_C1,sl_type_C2,sl_type_C4,sl_type_D2,sl_type_D3,sl_type_D4,...,sl_spec_10.0,sl_spec_16.0,sl_spec_22.0,sl_spec_25.0,sl_spec_35.0,sl_spec_38.0,sl_spec_60.0,sl_spec_70.0,sl_spec_100.0,sl_spec_120.0
19906,474220224975,1573B651,17,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
19907,474220224975,1573B651,11,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
19908,474220224975,1573B651,8,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
19909,474220224975,1573B661,20,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
19910,474220224975,1573B661,10,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
19911,474220224975,1573B664,21,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
19912,474220224975,1573B664,14,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
19913,474220224975,1573B664,6,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


#### 공사번호 기준 인입선 그룹 만들기

In [37]:
unique_cons_ids = df_training.cons_id.unique()
cons_id_row_values = []
columns_for_summation = df_training.columns.tolist()[2:]

In [38]:
# 지금까지 전처리된 데이터의 공사번호 갯 수와 인입선 데이터의 공사번호 갯 수 비교
df_cons.shape[0], len(unique_cons_ids)

# 인입선 공사가 없는 공사도 있음

(11383, 11209)

In [39]:
for cons_id in unique_cons_ids:
    df_temp = df_training[df_training.cons_id == cons_id]
    new_row = df_temp[columns_for_summation].sum().values.tolist()
    comp_id_count = df_temp.comp_id.nunique()
    cons_id_row_values.append([cons_id, comp_id_count, df_temp.shape[0]] + new_row)

In [40]:
df_group = pd.DataFrame(
    cons_id_row_values, 
    columns=['cons_id', 'sl_comp_id_cnts', 'real_sl_cnts', 'sl_span_sum'] \
        + columns_for_summation[1:]
)
df_group.shape

(11209, 27)

In [41]:
df_merge_sl = df_cons.merge(df_group, on='cons_id', how='left')
df_merge_sl.shape

(11383, 165)

In [42]:
df_merge_sl.isna().sum().sort_values(ascending=False)[:20]

sl_spec_120.0      174
sl_spec_1.6        174
real_sl_cnts       174
sl_span_sum        174
supervisor         174
sl_type_C1         174
sl_type_C2         174
sl_type_C4         174
sl_type_D2         174
sl_type_D3         174
sl_type_D4         174
sl_type_OW         174
sl_type_V2         174
sl_comp_id_cnts    174
sl_spec_2.6        174
sl_spec_25.0       174
sl_spec_100.0      174
sl_spec_70.0       174
sl_spec_60.0       174
sl_spec_35.0       174
dtype: int64

In [43]:
df_merge_sl.fillna(0.0, inplace=True)

In [44]:
write_data('1f-merge-cons-pole-line-sl-path', df_merge_sl, process_seq='4th')